In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Resistance/Support AND Candles Patterns

In [7]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/My Drive/My Drive/StokesData_Hackthon/RELIANCE.csv',parse_dates=['Date'],index_col = 'Date')

#Check if NA values are in data
df=df[df['Volume']!=0]

df.isna().sum()
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-08-23,2594.000000,2653.199951,2591.350098,2646.800049,2646.800049,4466619
2022-08-24,2640.100098,2670.000000,2635.350098,2639.750000,2639.750000,4785579
2022-08-25,2648.949951,2662.899902,2622.600098,2632.050049,2632.050049,3632902
2022-08-26,2633.000000,2650.000000,2607.000000,2618.000000,2618.000000,4957449
2022-08-29,2585.000000,2655.000000,2578.050049,2597.649902,2597.649902,9155684


# Support and Resitance FUNCTIONS

In [8]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.Low[i]>df1.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.Low[i]<df1.Low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.High[i]<df1.High[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.High[i]>df1.High[i-1]):
            return 0
    return 1

In [9]:
Length = len(df)
High = list(df['High'])
Low = list(df['Low'])
Close = list(df['Close'])
Open = list(df['Open'])
bodydiff = [0] * Length

Highdiff = [0] * Length
Lowdiff = [0] * Length
ratio1 = [0] * Length
ratio2 = [0] * Length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(Open[row]-Close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        Open[row-1]<Close[row-1] and
        Open[row]>Close[row] and 
        (Open[row]-Close[row-1])>=-0e-5 and Close[row]<Open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        Open[row-1]>Close[row-1] and
        Open[row]<Close[row] and 
        (Open[row]-Close[row-1])<=+0e-5 and Close[row]>Open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    Highdiff[row] = High[row]-max(Open[row],Close[row])
    Lowdiff[row] = min(Open[row],Close[row])-Low[row]
    bodydiff[row] = abs(Open[row]-Close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = Highdiff[row]/bodydiff[row]
    ratio2[row] = Lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and Lowdiff[row]<0.2*Highdiff[row] and bodydiff[row]>bodydiffmin):# and Open[row]>Close[row]):
        return 1
    elif (ratio2[row]>1 and Highdiff[row]<0.2*Lowdiff[row] and bodydiff[row]>bodydiffmin):# and Open[row]<Close[row]):
        return 2
    else:
        return 0
    
def CloseResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    C1 = abs(df.High[l]-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    C2 = abs(max(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    C3 = min(df.Open[l],df.Close[l])<min(levels, key=lambda x:abs(x-df.High[l]))
    C4 = df.Low[l]<min(levels, key=lambda x:abs(x-df.High[l]))
    if( (C1 or C2) and C3 and C4 ):
        return 1
    else:
        return 0
    
def CloseSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    C1 = abs(df.Low[l]-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    C2 = abs(min(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    C3 = max(df.Open[l],df.Close[l])>min(levels, key=lambda x:abs(x-df.Low[l]))
    C4 = df.High[l]>min(levels, key=lambda x:abs(x-df.Low[l]))
    if( (C1 or C2) and C3 and C4 ):
        return 1
    else:
        return 0

In [10]:
n1=2
n2=2
backCandles=45
signal = [0] * Length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and CloseResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and CloseSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [11]:
df['signal']=signal

In [12]:
df[df['signal']==1].count()

Open         14
High         14
Low          14
Close        14
Adj Close    14
Volume       14
signal       14
dtype: int64

In [13]:
SLTPRatio = 1.1 #TP/SL Ratio
def mytarget(barsupfront, df1):
    Length = len(df1)
    High = list(df1['High'])
    Low = list(df1['Low'])
    Close = list(df1['Close'])
    Open = list(df1['Open'])
    signal = list(df1['signal'])
    trendcat = [0] * Length
    amount = [0] * Length
    
    SL=0
    TP=0
    for line in range(backCandles, Length-barsupfront-n2):

        if signal[line]==1:
            SL = max(High[line-1:line+1])#!!!!! parameters
            TP = Close[line]-SLTPRatio*(SL-Close[line])
            for i in range(1,barsupfront+1):
                if(Low[line+i]<=TP and High[line+i]>=SL):
                    trendcat[line]=3
                    break
                elif (Low[line+i]<=TP):
                    trendcat[line]=1 #win trend 1 in signal 1
                    amount[line]=Close[line]-Low[line+i]
                    break
                elif (High[line+i]>=SL):
                    trendcat[line]=2 #loss trend 2 in signal 1
                    amount[line]=Close[line]-High[line+i]
                    break

        if signal[line]==2:
            SL = min(Low[line-1:line+1])#!!!!! parameters
            TP = Close[line]+SLTPRatio*(Close[line]-SL)
    
            for i in range(1,barsupfront+1):
                if(High[line+i]>=TP and Low[line+i]<=SL):
                    trendcat[line]=3
                    break
                elif (High[line+i]>=TP):
                    trendcat[line]=2 #win trend 2 in signal 2
                    amount[line]=High[line+i]-Close[line]
                    break
                elif (Low[line+i]<=SL):
                    trendcat[line]=1 #loss trend 1 in signal 2
                    amount[line]=Low[line+i]-Close[line]
                    break
    #return trendcat
    return amount

df['Trend'] = mytarget(16, df)
df['Amount'] = mytarget(16, df)

In [14]:
df[df['Amount']!=0]
#df['Amount'].sum()/18

,Open,High,Low,Close,Adj Close,Volume,signal,Trend,Amount
Date,,,,,,,,,
2018-11-13,1063.915161,1092.642822,1060.002197,1089.126221,1075.616089,6876165,2,51.759277,51.759277
2018-12-26,1075.604370,1089.671021,1055.643555,1088.036499,1074.540039,7278403,2,36.553467,36.553467
2019-02-07,1297.946777,1308.793945,1274.023560,1278.283203,1262.426758,9299549,1,39.030029,39.030029
2019-03-05,1211.912231,1228.158325,1207.157349,1226.028442,1210.820068,7189013,2,35.117188,35.117188
2019-07-04,1269.367676,1278.877563,1267.980835,1271.943237,1256.165405,4315672,1,23.774658,23.774658
2019-07-09,1237.222412,1271.447998,1233.804810,1268.079834,1252.349854,8092747,2,-37.048828,-37.048828
2019-08-08,1097.942627,1147.126343,1085.015137,1141.529419,1133.612427,14398134,2,149.037354,149.037354
2019-08-23,1227.365845,1271.943237,1214.983154,1263.869751,1255.104248,9833599,2,-54.334961,-54.334961
2019-09-12,1223.403320,1228.802124,1194.378540,1198.984863,1190.669434,5482620,1,37.346069,37.346069


In [15]:
import numpy as np
conditions = [(df['Trend'] == 1) & (df['signal'] == 1), (df['Trend'] == 2) & (df['signal'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

trendId=1
print(df[df['result']==trendId].result.count()/df[df['signal']==trendId].signal.count())
df[ (df['Trend']!=trendId) & (df['Trend']!=3) & (df['signal']==trendId) ] # false positives

0.0


,Open,High,Low,Close,Adj Close,Volume,signal,Trend,Amount,result
Date,,,,,,,,,,
2019-02-07,1297.946777,1308.793945,1274.023560,1278.283203,1262.426758,9299549,1,39.030029,39.030029,0
2019-07-04,1269.367676,1278.877563,1267.980835,1271.943237,1256.165405,4315672,1,23.774658,23.774658,0
2019-09-12,1223.403320,1228.802124,1194.378540,1198.984863,1190.669434,5482620,1,37.346069,37.346069,0
2020-04-24,1337.472046,1480.912476,1334.549805,1403.694336,1393.959106,62296159,1,-77.218140,-77.218140,0
2020-05-11,1565.163818,1599.835205,1557.536133,1561.993896,1551.160767,30959745,1,110.750244,110.750244,0
2020-11-25,1980.000000,1992.949951,1942.199951,1947.800049,1941.554199,15062376,1,-66.449951,-66.449951,0
2021-03-31,2018.000000,2049.899902,1999.000000,2003.099976,1996.676758,7499740,1,102.849976,102.849976,0
2021-07-23,2134.449951,2134.449951,2102.050049,2105.699951,2105.699951,4585938,1,34.599854,34.599854,0
2021-09-06,2413.000000,2480.000000,2412.000000,2425.600098,2425.600098,15525644,1,-71.399902,-71.399902,0


In [37]:
dfpl = df[-450
          :-400]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()

In [17]:
def SIGNAL():
    return df.signal

In [18]:
!pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - 75000e-4
            tp1 = self.data.Close[-1] + 60000e-4
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + 70000e-4
            tp1 = self.data.Close[-1] - 60000e-4
            self.sell(sl=sl1, tp=tp1)


def MovingAverage(closes:pd.Series, n:int) -> pd.Series:
    return pd.Series(closes).rolling(n).mean()
class SmaCross(Strategy):
    sma_fast = 12
    sma_slow = 24
    
    def init(self):
        self.sma1 = self.I(MovingAverage, self.data.Close, self.sma_fast)
        self.sma2 = self.I(MovingAverage, self.data.Close, self.sma_slow)

    def next(self):
        if not self.position and crossover(self.sma1, self.sma2):
            self.buy()
        elif self.position and crossover(self.sma2, self.sma1):
            self.position.close()

/usr/local/lib/python3.9/dist-packages/backtesting/_plotting.py:50: UserWarning:

Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.



In [40]:
from backtesting import Backtest

bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)
stat = bt.run()
stat

Start                     2018-08-01 00:00:00
End                       2022-08-29 00:00:00
Duration                   1489 days 00:00:00
Exposure Time [%]                    2.885572
Equity Final [$]                  8788.575122
Equity Peak [$]                       10000.0
Return [%]                         -12.114249
Buy & Hold Return [%]              119.924779
Return (Ann.) [%]                   -3.186089
Volatility (Ann.) [%]                1.776357
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.114249
Avg. Drawdown [%]                  -12.114249
Max. Drawdown Duration     1385 days 00:00:00
Avg. Drawdown Duration     1385 days 00:00:00
# Trades                                   29
Win Rate [%]                         6.896552
Best Trade [%]                       0.357631
Worst Trade [%]                     -2.183356
Avg. Trade [%]                    

In [35]:
bt.plot()

Row(id='2241', ...)

In [21]:
!pip install vectorbt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.4 MB/s eta 0:00:00
  Created wheel for vectorbt: filename=vectorbt-0.24.5-py3-none-any.whl size=526801 sha256=56b021916f2281aa91bdf20d2e9f51a90f0fb507f938f2eac38f08c5b0894f59
  Stored in directory: /root/.cache/pip/wheels/be/6a/39/9fe1027502bf84c884fcc956eaabdc6196d78daac11ce28f74
Successfully built vectorbt


In [ ]:
class SmaCross(Strategy):
    sma_fast = 12
    sma_slow = 24
    
    def init(self):
        self.sma1 = self.I(MovingAverage, self.data.Close, self.sma_fast)
        self.sma2 = self.I(MovingAverage, self.data.Close, self.sma_slow)

    def next(self):
        if not self.position and crossover(self.sma1, self.sma2):
            self.buy()
        elif self.position and crossover(self.sma2, self.sma1):
            self.position.close()

In [24]:
import vectorbt as vbt

price = df['Close']

fast_ma = vbt.MA.run(price, 10, short_name='fast')
slow_ma = vbt.MA.run(price, 20, short_name='slow')

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

In [26]:
pf = vbt.Portfolio.from_signals(price, entries, exits)

In [27]:
pf.total_return()

0.4915453960761981

In [32]:
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e79c60f9-6fd0-414a-a683-13d20503db9c',
              'x': array([datetime.datetime(2018, 8, 1, 0, 0),
                          datetime.datetime(2018, 8, 2, 0, 0),
                          datetime.datetime(2018, 8, 3, 0, 0), ...,
                          datetime.datetime(2022, 8, 25, 0, 0),
                          datetime.datetime(2022, 8, 26, 0, 0),
                          datetime.datetime(2022, 8, 29, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([1181.15380859, 1157.37915039, 1165.8984375 , ..., 2632.05004883,
                          2618.        , 2597.64990234]),
              'yaxis': 'y'},
             {'customdata': array([[ 0.        ,  0.09194216,  0.        ],
                                   [ 2.        ,  0.09013891,  0